In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath("__file__")))))
from nbafuns import *
import seaborn.objects as snso
from sklearn.cluster import KMeans

data_DIR = "../../data/synergy/"

In [ ]:
def get_playtypes(year, poss = 200):
    df = pd.read_parquet(data_DIR +f"NBA_Synergy_Player_PG_{year}.parquet")
    df.columns = map(str.lower, df.columns)
    dft = pd.read_parquet(data_DIR +f"NBA_Synergy_Player_Tot_{year}.parquet")
    dft.columns = map(str.lower, dft.columns)
    df1 = df.query("type_grouping == 'Offensive'")
    df1 = df1[["player_name","play_type","poss_pct"]]
    df1 = df1.sort_values(by=["player_name","play_type"])
    df1 = df1.fillna(0)
    dfs = df1.pivot_table(index= "player_name", columns= "play_type", values= "poss_pct", aggfunc= "mean")
    dfs = dfs.fillna(0)
    dfs = dfs.drop(columns= ["Misc","Transition"])
    dfs = dfs[['PRBallHandler','Isolation','Spotup','OffScreen','Handoff','Cut','PRRollMan','Postup','OffRebound']]
    dftp = dft.groupby("player_name")[["poss"]].agg({"poss":["sum"]})
    dftp.columns = ["poss"]
    dftp = dftp.sort_values(by=["player_name"])
    print(f"unfiltered length: {len(dftp)}")
    fil_len = len(dftp.query(f"poss > {poss}"))
    print(f"filtered   length: {fil_len}")
    dfs = dfs[dftp["poss"]>=poss]

    return dfs

In [ ]:
dfa = []
for season in range(2012,2023):
    dfs1 = get_playtypes(season, poss = 200)
    dfs1["Season"]  = season + 1
    dfs1 = dfs1.reset_index()
    dfa.append(dfs1)
clear_output()

In [ ]:
dfs = pd.concat(dfa)
dfs = dfs.reset_index(drop=True)
dfs.insert(1,"Season",dfs.pop("Season"))
features = dfs.iloc[:,2:].to_numpy()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
model = AgglomerativeClustering(n_clusters= 12, compute_distances =True, linkage= "ward")

model.fit(features)

In [ ]:
dfc = dfs.copy()
dfc["Ball"] = dfc["PRBallHandler"] + dfc["Isolation"]
dfc["Shoot"] = dfc["Spotup"] + dfc["OffScreen"] + dfc["Handoff"]
dfc["Big"] = dfc["Cut"] + dfc["PRRollMan"]	+ dfc["Postup"] + dfc["Postup"]
dfc["label"] = model.labels_
dfc.insert(2,"label",dfc.pop("label"))

In [ ]:
dfc

In [ ]:
i = 12
dfc.query(f"label == {i-1}").query("Season == 2023")